# Computer Simulation Project
Simulating a discrete-event CPU scheduler.

**Authors**: <br>
Mohammadreza Mofayezi 98106059 <br>
Amirhossein Bagheri 98105621

## Import Libraries

In [376]:
import simpy
from typing import List

import pandas as pd
import numpy  as np
from queue import PriorityQueue

from scipy.stats import norm, expon, poisson

## Inputs
The parameters, number of processes and duration of simulation.

In [377]:
# parameters
X = 8
Y = 12
Z = 100
# number of processes
N = 100
# length of simulation
L = 800
# task threshold
k = 10
seed = 1234
T1 = 2
T2 = 4

In [378]:
np.random.seed(seed=seed)

## Job Creator

In [379]:
class Task:
    def __init__(self, id: int):
        self.id = id
        self.inter_arrival = int(poisson.rvs(X))
        self.service_time = int(expon.rvs(Y))
        self.time_out = int(expon.rvs(Z))
        self.remaining_service = self.service_time
        self.priority = self.get_priority()
        self.arrival_time = None
        self.finish = -1
        self.time_outed = False
    def get_priority(self):
        return np.random.choice([1, 2, 3], 1, p=[0.1, 0.2, 0.7])[0]

    def __eq__(self, __o: object) -> bool:
        return self.priority == __o.priority

    def __repr__(self) -> str:
        return f'Task {self.id} Priority: {self.priority} - InterArrival: {self.inter_arrival} - ServiceTime: {self.service_time} FinishTime: {self.finish} SystemTime: {self.finish-self.arrival_time} WatingTime: {self.finish-self.arrival_time-self.service_time} Time_out: {self.time_out} TOuted: {self.time_outed}'

In [380]:
class JobCreator:
    def __init__(self, num_tasks: int):
        self.num_tasks = num_tasks
        self.tasks = PriorityQueue()
        self.all_tasks = []

    def run(self):
        start = 0
        task = Task(0)
        task.arrival_time = 0
        self.tasks.put((task.arrival_time,task.priority, -task.remaining_service, task))
        self.all_tasks.append(task)
        
        for i in range(1,self.num_tasks):
            task = Task(i)
            start += task.inter_arrival
            task.arrival_time = start
            self.tasks.put((task.arrival_time,task.priority, -task.remaining_service, task))
            self.all_tasks.append(task)

    def get_task(self,time: int):
        if self.tasks.queue[0][0] <= time:
            return self.tasks.get()[-1]
        return None
    
    def len(self):
        return len(self.tasks.queue)
    
    def empty_job(self):
        return self.tasks.empty()


In [381]:
class Que:
    def __init__(self):
        self.q = []
    
    def insert(self,task: Task):
        self.q.insert(0,task)
    
    def len(self):
        return len(self.q)
    
    def run(self,time: int):
        q = []
        for i in range(len(self.q)):
            if self.q[i].remaining_service > 0:
                q.append(self.q[i])
        self.q = q



    def get_task(self):
        return self.q[-1]
    
    def pop_task(self):
        return self.q.pop()

    def give_up(self, task: Task = None):
        return False

    def timesim(self):
        return float("inf")



class RR(Que):
    def __init__(self, T: int):
        super().__init__()
        self.T = T
    
    def give_up(self, task: Task = None):
        return task.remaining_service > self.T
    
    def timesim(self):
        return self.T

class Processor:
    def __init__(self, num_tasks: int, maxsimul: int, queues :List, P: List):
        self.Job_creator = JobCreator(num_tasks)
        self.Job_creator.run()
        self.queues = queues
        self.simultime = 0
        self.max_simultime = maxsimul
        self.K = 10
        self.order_of_finish = []
        self.P = P
    
    def job_loader(self, k: int):
        if sum([q.len() for q in self.queues]) >= k:
            return
        queue = PriorityQueue()
        for _ in range(k):
            if self.Job_creator.empty_job():
                break
            task = self.Job_creator.get_task(self.simultime)
            if task is None:
                break

            queue.put((task.priority,-task.remaining_service,task))
        
        for q in queue.queue:
            self.queues[0].insert(q[-1])
    
    def process(self, i: int):
        q = self.queues[i]
        task = q.get_task()
        give_up = q.give_up(task)
        time = min(q.timesim(),task.remaining_service)
        task.remaining_service = task.remaining_service - time
        self.simultime += time
        if task.remaining_service == 0:
            self.order_of_finish.append(task)
            task.finish = self.simultime
        q.run(self.simultime)
        if give_up :
            task = q.pop_task()
            self.queues[i+1].insert(task)
        self.job_loader(self.K)
    
    def queue_chooser(self):
        return np.random.choice(list(range(len(self.P))), 1, p=self.P)[0]
        
    def time_out(self):
        for q in self.queues:
            qq = []
            for i in range(q.len()):
                if self.simultime - q.q[i].arrival_time > q.q[i].time_out:
                    q.q[i].finish = self.simultime
                    q.q[i].time_outed = True
                    self.order_of_finish.append(q.q[i])
                else:
                    qq.append(q.q[i])
            q.q = qq
        

    def dispatcher(self):
        while (self.Job_creator.len() > 0 or sum([q.len() for q in self.queues]) > 0) and self.simultime <= self.max_simultime:
            if sum([q.len() for q in self.queues]) > 0:
                choose = self.queue_chooser()
                if self.queues[choose].len() > 0:
                    self.process(choose)

            else:
                print(self.simultime)
                self.job_loader(self.K)
            self.time_out()
            

    
    def run(self):
        self.job_loader(self.K)
        self.dispatcher()

        

processor = Processor(N, L, [RR(T1),RR(T2),Que()],[0.8,0.1,0.1])
processor.run()
print(f"number of jobs Done {len(processor.order_of_finish)}")
for i in processor.order_of_finish:
    print(i)


        

number of jobs Done 86
Task 0 Priority: 3 - InterArrival: 11 - ServiceTime: 13 FinishTime: 13 SystemTime: 13 WatingTime: 0 Time_out: 101 TOuted: False
Task 1 Priority: 3 - InterArrival: 7 - ServiceTime: 12 FinishTime: 55 SystemTime: 48 WatingTime: 36 Time_out: 102 TOuted: False
Task 2 Priority: 3 - InterArrival: 10 - ServiceTime: 14 FinishTime: 65 SystemTime: 48 WatingTime: 34 Time_out: 100 TOuted: False
Task 3 Priority: 3 - InterArrival: 4 - ServiceTime: 12 FinishTime: 79 SystemTime: 58 WatingTime: 46 Time_out: 100 TOuted: False
Task 4 Priority: 3 - InterArrival: 5 - ServiceTime: 12 FinishTime: 87 SystemTime: 61 WatingTime: 49 Time_out: 102 TOuted: False
Task 5 Priority: 2 - InterArrival: 8 - ServiceTime: 12 FinishTime: 105 SystemTime: 71 WatingTime: 59 Time_out: 104 TOuted: False
Task 6 Priority: 3 - InterArrival: 7 - ServiceTime: 12 FinishTime: 111 SystemTime: 70 WatingTime: 58 Time_out: 100 TOuted: False
Task 7 Priority: 3 - InterArrival: 11 - ServiceTime: 12 FinishTime: 123 System

## Scheduler

In [382]:
class Scheduler:
    def __init__(self, env):
        self.env = env
        simpy.Resource(env, )

    def job_loader(self):
        pass

    def dispatcher(self):
        pass